In [ ]:
from pyspark.sql.functions import *
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.feature import Tokenizer
import pyspark.sql.functions as func

In [ ]:
data =spark.read.option("header", "true").\
    option("mode", "DROPMALFORMED")\
    .csv("Reviews_1400.csv")


In [ ]:
# todo: delete
data.describe('Summary')\
    .show()

data.describe('Score')\
    .show()

data.printSchema()

data.count()


In [ ]:
data = data.select('Score', 'Summary')\
    .dropna()
data.count()

In [ ]:
data = data.where(length(col('Summary')) >= 75)


In [ ]:
# todo: delete
data.count()


In [ ]:
def filterData(column):

    return trim(lower(regexp_replace(column, '[^\sa-zA-Z0-9]', ''))).alias('FilteredColumn')

filteredData = data.select([filterData(data['Summary']), col("Score").alias("label")])

# נבחר לעשות  שלילי וחיובי
filteredData = filteredData.withColumn("label", when(col("label") > 80, 1.0).otherwise(0.0))

